# Load and initialize

settings

In [ ]:
# dir="~/Desktop/biology/breast_cancer/data/"
# ubunto:
# old samples
dir1="~/Dropbox/CyTOF_Breast/Kaplan_1st/"
# new samples
dir2 = '~/Dropbox/CyTOF_Breast/Kaplan_2nd/data_afterGating/processed data/'
# plot dir
dir_plots = 'Plots_preproccess/'

# home:
# old samples
# dir1="C:/Users/yishai/Dropbox/CyTOF_Breast/Kaplan_1st/"
# new samples
# dir2 = "C:/Users/yishai/Dropbox/CyTOF_Breast/Kaplan_2nd/data_afterGating/processed data/"
# plot dir
# dir_plots = 'C:/Users/yishai/Desktop/biology/breast_cancer/Plots'

# plots:
    # create visualization
visualize = True
    # show figures
show = False

subsample = False



# plt.savefig("test.svg", format="svg")




envirement

In [ ]:


#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 22 20:09:58 2021

@author: ronguy
"""
import time 
start = time.process_time()
import os

import sys
from IPython.display import Image, display
# from tqdm import tqdm_notebook,tqdm
# import time

import numpy as np
# import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
# import matplotlib.patches as  mpatches
plt.rcParams["figure.figsize"] = (5.0, 4.0)  # Set default size of plots.
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 180) #according to screen width
# from pandas.core.base import PandasObject
# PandasObject.view = view #allows view meth pd.view

from scipy import signal, stats
import seaborn as sns

from sklearn import metrics
from sklearn.cluster import KMeans, DBSCAN, MiniBatchKMeans
# from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
# from sklearn.metrics import silhouette_samples,silhouette_score
# from sklearn.neighbors import kneighbors_graph

from lmfit import minimize, Parameters
# from umap import UMAP

import scanpy as sc
import anndata

# import schist as scs

# import networkx as nx

# from castle.common import GraphDAG
# from castle.metrics import MetricsDAG
# from castle.datasets import IIDSimulation, DAG
# from castle.algorithms import PC,Notears,GOLEM,ANMNonlinear,DirectLiNGAM,ICALiNGAM,NotearsLowRank
# import notears.notears as notears

# from shapely.geometry import Point
# from shapely.geometry.polygon import Polygon

# import keras
# from keras import layers
# from keras.callbacks import EarlyStopping
# from keras.callbacks import ModelCheckpoint

# from statsmodels.graphics._regressionplots_doc import _plot_influence_doc
# from statsmodels.regression.linear_model import OLS
# from statsmodels.stats.outliers_influence import variance_inflation_factor



from usefull_functions import *
from functions import *
from normalization import *


%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

folderExists(dir_plots)

Feature names (Participating panel)


Received CyTOF data is assumed to be only of intact single and alive cells  

each sample matrix with high dimensionality feature space (NamesAll) and without duplicate samples

experiment2 without: 'p53', 'ZEB1', 
experiment1 without: 'NCad','ECad','panKeratin'



In [ ]:
features = {}
features['NamesAll'] = [  
            'CD45','H4','H3', 'H3.3','NCad','ECad','panKeratin', 'K5', 'EpCam', 'H3K27me2', 'p53', 'EZH2', 'H3K4me3', 'gH2AX','aSMA','H3K36me2','H3K4me1','H3K9me2','H4K16ac',
            'H2Aub', 'Vimentin', 'H3K64ac', 'BMI-1', 'ZEB1',  'H3K27ac', 'H4K20me3', 'ER', 'CD49f', 'H3K36me3', 'CD24',
            'GATA3', 'H3K27me3', 'H3K9ac', 'H3K9me3', 'CD44', 'Ki67', 'K8-18', 'H3S28p', 'Ir_DNA2', 'Live_Dead',
        ]

# extra cellular are outside the cell - except them everything is noramalized (in group "ToNorm")
features['ToNorm'] = [    
            'H4','H3', 'H3.3','panKeratin', 'K5', 'H3K27me2', 'p53', 'EZH2', 'H3K4me3', 'gH2AX', 'aSMA', 'H3K36me2', 'H3K4me1', 'H3K9me2',
            'H4K16ac', 'H2Aub', 'Vimentin', 'H3K64ac', 'BMI-1', 'ZEB1', 'H3K27ac', 'H4K20me3',
            'ER', 'H3K36me3', 'GATA3', 'H3K27me3', 'H3K9ac', 'H3K9me3', 'Ki67', 'K8-18', 'H3S28p',
        ]
features['CellIden'] = [  
            'CD45', 'K5', 'EpCam', 'aSMA', 'Vimentin', 'ZEB1', 'ER', 'CD49f', 'CD24', 'GATA3', 'CD44', 'K8-18',
            'Ki67','p53',
            'NCad','ECad','panKeratin',
        ]
# everything that is not cell identity
features['EpiCols'] = [   
            'H4','H3', 'H3.3', 'H3K27me2', 'H3K4me3', 'H3K36me2', 'H3K4me1', 'H3K9me2', 'H4K16ac', 'H2Aub', 'H3K64ac',  
            'H3K27ac', 'H4K20me3', 'BMI-1','EZH2','gH2AX',
            'H3K36me3', 'H3K27me3', 'H3K9ac', 'H3K9me3', 'H3S28p'
        ]

# intra cellular??
features['NMS'] = [       
            'CD45', 'K5', 'EpCam', 'H3K27me2', 'p53', 'EZH2', 'H3K4me3', 'gH2AX', 'aSMA', 'H3K36me2', 'H3K4me1',
            'H3K9me2', 'H4K16ac', 'H2Aub', 'Vimentin', 'H3K64ac', 'BMI-1', 'ZEB1', 'H3K27ac', 'H4K20me3', 'ER',
            'CD49f', 'H3K36me3', 'CD24', 'GATA3', 'H3K27me3', 'H3K9ac', 'H3K9me3', 'CD44', 'Ki67', 'K8-18', 'H3S28p',
        ]

features = removeFeatures(features.copy(),features =['Ir_DNA2', 'Live_Dead'])
# diffrence of features1
features1 = removeFeatures(features.copy(),features =['NCad','ECad','panKeratin',])
# diffrence of features2
features2 = removeFeatures(features.copy(),features =['p53','ZEB1'])


    #  'H3S28p','p53', 'EZH2', 'gH2AX', 'BMI-1', 'Ki67', 'Ir_DNA2', 'Live_Dead'


names = {}
names['figures'] = [1,1,2,2,2]

names['1'] = features1
names['2'] = features1
names['3'] = features2
names['4'] = features2
names['5'] = features2
names['all'] = features
del features;del features1; del features2


def test_fetures(NamesAll,CellIden,EpiCols):
 
  new_NamesAll = CellIden+EpiCols
  test1 = all(item in new_NamesAll for item in NamesAll)
  if not test1:
    for i in new_NamesAll:
      NamesAll.remove(i)
      
    print(NamesAll)
    
  test2 = not (any(item in CellIden for item in EpiCols))
  return test1*test2
 
 

# verify:
#   CellIden +EpiCols = NamesAll
#   no feature is both in CellIden and EpiCols
for i, name in names.items():
  try:
    NamesAll = name['NamesAll'].copy()
    CellIden = name['CellIden'].copy()
    EpiCols = name['EpiCols'].copy()
    if test_fetures(NamesAll,CellIden,EpiCols) != 1:
      print('error in sample', i)
  except:
    pass
print ('features are correct!')
  

      

Load data from csv, and store in dictionary 

In [ ]:
K1=pd.read_csv(dir1+"BCK-01_noaf_18Sep2022_01_0.fcs_file_internal_comp_residual.csv")
K2=pd.read_csv(dir1+"BCK-02_noaf_18Sep2022_01_0.fcs_file_internal_comp_residual.csv")

# new features - without:p53, zeb1, with: pankeratin, ecad, ncad 

K3=pd.read_csv(dir2+"export_BCK03_noaf_23Nov2022_01_0_final_cells - processed.csv")
K4=pd.read_csv(dir2+"export_BCK04_noaf_23Nov2022_02_0_final_cells - processed.csv")
K5=pd.read_csv(dir2+"export_BCK05_noaf_23Nov2022_01_0_final_cells - processed.csv")

# later analysis k6 with duplicates
# K6=pd.read_csv(dir2+"export_BCK05_noaf_23Nov2022_01_0_final_cells.csv")






k={}
k['1'] = K1[names['1']['NamesAll']]
k['2'] = K2[names['2']['NamesAll']]

k['3'] = K3[names['3']['NamesAll']]
k['4'] = K4[names['4']['NamesAll']]
k['5'] = K5[names['5']['NamesAll']]

del K1;del K2;del K3;del K4;del K5




Visualize data

In [ ]:
if show:
  for i, K in k.items():
      print ('k'+i)
      print ('\n')
  #     print(K.head(3)) #see samples
      print(K.describe()) #mean,std, max,min, q25,q50,q75
      print ('\n')

# gating and transformation



gating - fix fluidigm gating result; In-gate and outlier removal (Samples with low number of events)

Samples with low number of events are discarded (outliers) - remove outlier 99.99% from all

In gate to achieve good normalization - gate on H3.3/H4 was too low


In [ ]:


def Gate(data,name,GateColumns):
    ddf=data.copy()
    print(name, 'gating on: ', GateColumns)
    print("Initial number of samples: ",len(ddf))
    # index of gating values ddf[['H3.3','H4']] larger than  Core gate value(5) 
    ind = (ddf[GateColumns]>5).all(axis=1)   
    ddf=ddf[ind]
    print("Core Gate: ",len(ddf))
    
    # index of quantile 99.99% gating values (remove outliers)
    quantile = np.quantile(ddf,0.9999,axis=0)
    outliers = ddf[(ddf>quantile).any(axis=1)]
    ddf=ddf[(ddf<quantile).all(axis=1)]
    print("Outlier Gate: ",len(ddf), ', samples removed:', len(outliers))
    # outliers distribution is not in a specific feature:
    
    # outliers [ outliers>quantile] = None
    # print(outliers.head(35))
    data=ddf.copy()
    del ddf
    return data


GateColumns=['H3.3','H4']#,'H3']#,'H3']
for [i, K] , j in zip(k.items(), names['figures']):
  
    if j==1:
      k[i] = Gate(K,'k' + i,['H3.3','H4'])
      print ('k' + i, ': gated')
    if j==2:
      k[i] = Gate(K,'k' + i,['H3','H3.3','H4'])
      print (f'k{i}: gated with method {j}')
    

# k_gate = k

 arcsinh data transformation
  
  transformation (semi log scaling): scale argument = 5

In [ ]:
def arcsinh_transform(unscaled_data, scale=5):
    scaled_data=np.arcsinh(unscaled_data.copy()/scale)
    return scaled_data
    

for i, K in k.items():
    k[i] = arcsinh_transform(K)
    print ('k' + i,'; arcsinh transformed')
    


describe data

In [ ]:
if show:
  for i, K in k.items():
      print ('k'+i)
      
      # print(K.head(3)) #see samples
      print(K.describe()) #mean,std, max,min, q25,q50,q75
      print ('\n')

visualization

arcsinh probability density plot - analysis of herogeneity (division to multiple populations), results:

cd45 - we want only the negative population

2 close populations - epcam, vimentin,cd49f

2 far populations - cd44

In [ ]:
# # Kernel Distribution Estimation Plot (kdeplot) is the probability density function plot
# # can plot for the univariate (single variable) or multiple variables altogether. 
# # i.e y = probability, x = feature value after arcsinh transformation (can be also negative)
if visualize:
  interst=[  'CD45','CD44', 'EpCam', 'Vimentin', 'CD49f','CD45','H4','H3', 'H3.3','NCad','ECad','panKeratin',]
  plot_hist(k,interst,names['figures'],dir_plots,show,
          func = sns.kdeplot,title = 'ArcSinh Unnormalized',figname = '1_Hist_ArcSinh_Unnormalized_' )  

In [ ]:
if visualize:
  plot_hist(k,names['all']['NamesAll'],names['figures'],dir_plots,show,
            func = sns.kdeplot,title = 'ArcSinh Unnormalized',figname = '1_Hist_ArcSinh_Unnormalized_' ) 

# Split CD45 neg 

the only interesting population

In [ ]:
# Kernel density estimation is the random variable's probability density function (PDF) estimatation
def inverse_PDF(K,i,feature, min_x = 'None'):
  
    # get data value for data with highest probability (peak)
      
  kernel = stats.gaussian_kde(K[feature])
  r=np.linspace(K[feature].min(),5,100)
  if min_x =='None':
    # avoid zero division using max; # inv_data = 1/np.maximum(1e-9,kernel(r))
    inv_data = 1/kernel(r)

    # minima : use builtin function fo find (min) peaks (use inversed data)
    # get index of input vector peaks, widths input - expected width of peaks of interest
    # min_peakind = signal.find_peaks_cwt(inv_data,np.arange(0.02,20,0.001))
    min_peakind = signal.find_peaks_cwt(inv_data,np.arange(1,100))

    min_x = r[min_peakind[0]]
  KCD45Neg = K[K[feature]<min_x].copy()
  min_y = kernel(min_x)[0]
  return KCD45Neg ,min_x, min_y



kCD45Neg ,min_x, min_y = {},{},{}
for i, K in k.items():
    
    try:
      kCD45Neg[i], min_x[i],min_y[i] = inverse_PDF(K,i,'CD45')
      print ('k' + i+'; CD45 neg seperated')
      print('value with highest *inverse* probability:', min_x[i])
    except:
        print('k' + i,';minima not found')
print('manual')        
kCD45Neg['4'], min_x['4'],min_y['4'] = inverse_PDF(k['4'],'4','CD45',min_x = 0.5)
print ('k4; CD45 neg seperated')
print('value with highest *inverse* probability:', min_x['4'])

if visualize:
  for i, K in k.items():
    plt.figure(figsize=(9,3))
    plt.title(f'K{i} ; limit = {np.round(min_x[i],4)}')
    sns.kdeplot(K['CD45'])
    plt.scatter(min_x[i],min_y[i])
    plt.savefig(dir_plots+'/K'+i+'_cd45_split.svg', format="svg", bbox_inches="tight", pad_inches=0.2)
    # plt.savefig(dir_plots+'/K'+i+'_cd45_split.png',dpi=200,bbox_inches='tight')
del min_x; del min_y

# # ThK1={'CD45': 1.8686868686868687}
# # ThK2={'CD45': 2.1717171717171717}


In [ ]:
if visualize:
  colors = cm.rainbow(np.linspace(0, 1, len (k.keys())))
  fig, ax = plt.subplots(1,2,figsize=(10,3))
  for i, K in k.items(): 
    #   
    j =  names['figures'][int(i)-1]-1 
    sns.histplot(K.CD45,color=colors[int(i)-1],label='T'+i,stat='density',element='step',fill=False,ax = ax[j])
    ax[j].legend(loc='upper center',bbox_to_anchor=(1,1))
    ax[j].set_yscale('log')
    ax[j].title.set_text('cd45 split histplot')
  plt.savefig(dir_plots+'cd45_split_histplot.svg', format="svg", bbox_inches="tight", pad_inches=0.2)

 


  fig, ax = plt.subplots(1,2,figsize=(12,3))
  for [i, K],[h, KCD45Neg] in zip( k.items(),kCD45Neg.items()):
      j =  names['figures'][int(i)-1]-1 
      sns.kdeplot(K.CD45,color=colors[int(i)-1],label='T'+i,ax = ax[j])
      sns.kdeplot(K.CD45,color=colors[int(i)-1],ls='--',label='T'+i+' CD45-',ax = ax[j])
      ax[j].legend(loc='upper center',bbox_to_anchor=(1,1))
      ax[j].set_yscale('log')
      ax[j].title.set_text('cd45 split kdeplot')
  plt.savefig(dir_plots+'cd45_split_kdeplot.svg', format="svg", bbox_inches="tight", pad_inches=0.2)

      

Continue only with CD45 neg population

1 - delete cd45+ population and continue with k  = kCD45Neg

2 - delete cd45 feature from data 

In [ ]:
del k
k = kCD45Neg.copy()
del kCD45Neg

for i, K in k.items():
  K.drop( columns='CD45')
  k[i] = K
  
  # remove from features lists(names)
  N = names[i]
  for j, sublist in N.items():
    try:
      sublist.remove('CD45')
    except:
      pass
    N[j] = sublist
  names[i] = N



# Normalize using new method on all intercellular markers

2 step normalization:

1- normalize gardient (TBD)

1- normalize using core

insight: since its the minima multiple rus with same effect as one run

note: features to norm (i.e names[i]['ToNorm']) are different on each sample

In [ ]:
# if visualize:
#   for i, K in k.items():
#     print ('k' + i,', initial std:')
#     print(K.std().to_frame().T)




def normalize_data(k,sample,names):
    for [i, K] , j in zip(k.items(), names['figures']):
        
        # FIRST STEP - normalize_gardient by core 
        # SECOND STEP - normalize ToNorm fetures by the core features ['H3.3','H4']
        
        if j==1:
          K=NormalizeNew2(K,names[i]['ToNorm'])
          K = Mean_Core_normalization(K, names[i]['ToNorm'],coreFetures=['H3.3','H4'])
        elif j==2:
          K=NormalizeNew(K,names[i]['ToNorm'])
          K = Mean_Core_normalization(K, names[i]['ToNorm'],coreFetures=['H4','H3', 'H3.3'])
        else: 
          print ('error')
          break
        
        k[i] = K 
        print (sample + i,f'; normalized with method {j}')
    return k
k = normalize_data(k,'k',names)



Scale data 

using mean, std (whiten data) - achieve std close to 1 mean close to 0

In [ ]:

for i, K in k.items():
    k[i] = scale_data(K)
    # print(K.std().to_frame().T)
    



In [ ]:


if visualize:
  plot_hist(k,names['all']['NamesAll'],names['figures'],dir_plots,show,
            func = sns.kdeplot,title = 'Normalized + Scaled',figname = '2_Hist_normalized_scaled_' )     

Subsample data

verify; n<=5000 

In [ ]:

# if subsample:
#   k = subsample_data(k,'k',n=5000)
# else:
  # for i, K in k.items():
  #   print ('k'+i+ '; samples = ', len(K), ', fetures = ', len(K.columns))
    
for i, K in k.items():
  print ('k'+i+ '; samples = ', len(K), ', fetures = ', len(K.columns))

            


## other
Save data to file (pickle)

convert images to pdf


In [ ]:


for i, K in k.items():
  dict ={}
  dict['k'] = K
  dict['names'] = names[i]
  dict['all'] = names['all']
  pickle_dump('k'+i+'_dict', dict)
  print('k'+i,'; saved to file')



del k;  del names

# imList2pdf(dir_plots)


end = time.process_time()
print ('total run time =', end-start )